In [1]:
#Install all dependency packages for the course
#Remember to execute this before running any of the exercises
!pip install tenacity==9.0.0
!pip install langchain==0.3.12
!pip install langchain-openai==0.2.12
!pip install langchain_community==0.3.12
!pip install langgraph==0.2.59
!pip install pysqlite3-binary==0.5.4
!pip install langchain_chroma==0.1.4
!pip install pandas==2.2.3
!pip install pypdf==5.1.0
!pip install nbformat==5.10.4
!pip install -U langchain-core langchain-google-genai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.3/770.3 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 52.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 29.9 MB/s eta 0:00:0000:0

## 2.3. Setup Function Tools for ReAct Agent

In [1]:
from langchain_core.tools import tool

#Tool annotation identifies a function as a tool automatically
@tool
def find_sum(x:int, y:int) -> int :
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return their sum.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y

@tool
def find_product(x:int, y:int) -> int :
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers as inputs and returns an integer as ouput.
    """
    return x * y


## 2.4. Create a basic ReAct Agent

In [2]:
from langchain_openai import AzureChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
import os
#Setup the LLM for the agent

#API info. Replace with your own keys and end points
os.environ["AZURE_OPENAI_API_KEY"]="4e4ab31800a64ae892cbb768fe28c0fc"
os.environ["AZURE_OPENAI_ENDPOINT"]="https://agentic-ai-course-kumaran.openai.azure.com/"

#Setup the LLM
'''
model = AzureChatOpenAI(
    azure_deployment="gpt-4o" ,
    api_version="2023-03-15-preview",
    model="gpt-4o"
)
'''

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    print("❌ ERROR: GOOGLE_API_KEY not found in environment variables.")
    # Stop execution so you don't waste time debugging
    sys.exit(1)
#Test the model
#response = llm.invoke("Hello, how are you?")
#print(response.content)


In [12]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create list of tools available to the agent
agent_tools=[find_sum, find_product]

#System prompt
system_prompt = SystemMessage(
    """You are a Math genius who can solve math problems. Solve the
    problems provided by the user, by using only tools available. 
    Do not solve the problem yourself"""
)

agent_graph=create_react_agent(
    model=model, 
    state_modifier=system_prompt,
    tools=agent_tools)


## 2.5. Execute the ReAct Agent

In [13]:
#Example 1
inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : The sum of 2 and 3 is 5. 

Step by Step execution : 
================================ Human Message =================================

what is the sum of 2 and 3 ?
================================== Ai Message ==================================
Tool Calls:
  find_sum (6e89db5e-b58c-4814-a216-99e335517bee)
 Call ID: 6e89db5e-b58c-4814-a216-99e335517bee
  Args:
    x: 2
    y: 3
================================= Tool Message =================================
Name: find_sum

5
================================== Ai Message ==================================

The sum of 2 and 3 is 5.


In [14]:
#Example 2
inputs = {"messages":[("user","What is 3 multipled by 2 and 5 + 1 ?")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

Agent returned : 3 multiplied by 2 is 6, and 5 + 1 is 6. 

Step by Step execution : 
================================ Human Message =================================

What is 3 multipled by 2 and 5 + 1 ?
================================== Ai Message ==================================
Tool Calls:
  find_product (4f5ff003-5ae2-4dbd-bf9e-3eeda9eb2213)
 Call ID: 4f5ff003-5ae2-4dbd-bf9e-3eeda9eb2213
  Args:
    y: 2
    x: 3
  find_sum (b73c17e3-2ae5-43a7-b960-35e4b47ce44e)
 Call ID: b73c17e3-2ae5-43a7-b960-35e4b47ce44e
  Args:
    x: 5
    y: 1
================================= Tool Message =================================
Name: find_product

6
================================= Tool Message =================================
Name: find_sum

6
================================== Ai Message ==================================

3 multiplied by 2 is 6, and 5 + 1 is 6.


## 2.6. Debugging the Agent

In [15]:
agent_graph=create_react_agent(
    model=model, 
    state_modifier=system_prompt,
    tools=agent_tools,
    debug=True)

inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user', 'what is the sum of 2 and 3 ?')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', 'what is the sum of 2 and 3 ?')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='57e5be1e-26b3-4b7e-b4e4-3947272eae52')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='57e5be1e-26b3-4b7e-b4e4-3947272eae52')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='', additional_kwargs={'function_call': {'name': 'find_sum', 'arguments': '{"x": 2, "y": 3}'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safet